# Laboratorio 1 - Analisis de ventas Big Data - Ivan Gonzalo Tapia



### Consignas:
Preguntas a responder en el análisis
—————————————————————————————————————————
1. Comportamiento de las ventas en los distintos meses:

  ● ¿Cómo variaron las ventas a lo largo de los diferentes meses?

  ● ¿Hubo algún mes que se destacó en términos de ventas?

  ● ¿Cuál es el ingreso total generado por mes?
2. Optimización de la publicidad y patrón de ventas por hora:

  ● ¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la
  probabilidad de compra?

  ● ¿Cómo cambian los patrones de ventas por hora a lo largo del año?

  ● ¿Hay modificaciones en los patrones de ventas durante las horas de mayor actividad
  en los distintos meses?

3. Distribución de ventas por ubicación:

  ● ¿En qué ciudades se han registrado las mayores ventas?

  ● ¿Cómo se comparan las ventas en diferentes estados o regiones?

  ● ¿Existe variación en las ventas por estado a lo largo de los meses?

4. Análisis del producto más vendido:

  ● ¿Cuál es el producto más vendido en general y en cada mes?

  ● ¿Qué factores crees que han contribuido al éxito de ese producto en particular?

5. Tendencia ventas:

  ● ¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días
  en los diferentes meses?

  ● ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

  ● ¿Se observa alguna diferencia significativa en las ventas entre los días laborables y
  los fines de semana?

6. Impacto de eventos especiales en las ventas:

  ● ¿Se ha observado algún aumento o disminución significativa en las ventas en días
  cercanos a eventos especiales, como días festivos?

  ● ¿Qué eventos específicos han tenido un impacto notable en el comportamiento de
  las ventas y cómo se manifestó ese impacto?

Pregunta Adicional
—————————————————————————————————————————

   - Además de las preguntas mencionadas anteriormente, te invito a pensar en una pregunta
  adicional que podría ser relevante para analizar los datos de ventas mensuales. Esta
  pregunta debe estar basada en la información que se proporciona en los conjuntos de
  datos. Puede estar relacionada con tendencias, comparaciones, patrones o cualquier otro
  aspecto que despierte tu interés. Trata de ser creativo/a y pensar en cómo podrías explorar
  aún más estos datos!


Pasos del Análisis
—————————————————————————————————————————
1. Carga de datos:
  - Cargar los datos de ventas de cada archivo CSV.
2. Limpieza de los datos:
  - Eliminar datos no numéricos.
  - Eliminar filas incompletas
3. Preparación de los datos:
  - Ajustar los tipos de datos de cada atributo
  - Extraer características importantes para el análisis como meses, horas, ciudades etc.
4. Análisis de datos:
  Ejemplo:
  - Examinar las horas del día en que las ventas son más frecuentes.
  - Determinar el mejor momento para mostrar publicidad.
5. Visualización de datos:
  - Utilizar gráficos y visualizaciones para representar los resultados de los análisis.
  - Crear gráficos de barras, líneas, mapas, tablas, etc, según corresponda.

## Import de librerias y recursos a utilizar

In [1]:
# Librerias necesarias para el desarrollo del proyecto
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


In [2]:
# Directorio propio para importar los datasets necesarios

path = '/content/drive/MyDrive/UPSO ML & BD/Dataset de ventas2'

archivos = os.listdir(path)
archivos

['Dataset_de_ventas_Abril.csv',
 'Dataset_de_ventas_Agosto.csv',
 'Dataset_de_ventas_Diciembre.csv',
 'Dataset_de_ventas_Enero.csv',
 'Dataset_de_ventas_Febrero.csv',
 'Dataset_de_ventas_Julio.csv',
 'Dataset_de_ventas_Junio.csv',
 'Dataset_de_ventas_Marzo.csv',
 'Dataset_de_ventas_Mayo.csv',
 'Dataset_de_ventas_Noviembre.csv',
 'Dataset_de_ventas_Octubre.csv',
 'Dataset_de_ventas_Septiembre.csv']

## Preparacion y limpieza del DataFrame

In [5]:
# Ahora voy a crear un diccionario para guardar estos archivos(datasets), para luego hacer un df

dataframes = {}

for archivo in archivos:
  if archivo.endswith('csv'):
    mes = archivo.split('_')[3].split('.')[0] # obtener el nombre del mes solo
    #aca obtenemos la ruta completa de donde se encuentra el csv de cada mes
    path_completo = os.path.join(path, archivo) # obtener ruta de acceso completa
    df = pd.read_csv(path_completo)   #cargar el archivo csv en un df
    #vamos a trabajar sobre una copia del df original
    df_ventas = df.copy()


    # SANITIZAR CADA DF ANTES DE CARGARLO
    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors = 'coerce') #errors = coerce transfora lo
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors = 'coerce')
    #esta es una mascara que me genera un true si hay un nan, y sino hay nan me pone un false
    df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]
    #aca elimino los nan's
    df_ventas = df_ventas.dropna()
    #aca voy a pasar todos los datos al tipo q corresponda y agregar las columnas que sean necesarias
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int) #esta funcion me transforma todos los valores a numericos a int
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(float) #esta funcion me transforma todos los valores a numericos a float

    # FINALIZA SANITIZACION

    # CREO COLUMNAS NECESARIAS PARA ANALISIS
    df_ventas['Fecha de Pedido'] = pd.to_datetime(df_ventas['Fecha de Pedido'], format='%m/%d/%y %H:%M')

    # Obtengo la hora y minutos como numero entero de 4 dígitos (por ejemplo, 1430)

    df_ventas['Estado'] = df_ventas['Dirección de Envio'].str.split(', ').str[-1].str.split(' ').str[0]
    df_ventas['Ciudad'] = df_ventas['Dirección de Envio'].str.split(', ').str[1]
    df_ventas['Hora'] = df_ventas['Fecha de Pedido'].dt.strftime('%H').astype(int)
    df_ventas['Dia'] = df_ventas['Fecha de Pedido'].dt.strftime('%d').astype(int)
    df_ventas['Mes'] = df_ventas['Fecha de Pedido'].dt.strftime('%m').astype(int)
    #FINALIZA CREACION DE COLUMNAS

    #aca sucede la magia, vamos agregando cada mes al df ventas
    #a cada mes le cargo su dataset correspondiente, por eso uso la variable mes
    dataframes[mes] = df_ventas

# Ordeno los meses
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']


In [ ]:
# Agarro un df y muestro el top
dataframe_e = dataframes['Enero']
dataframe_e.head()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Estado,Ciudad,Hora,Dia,Mes
0,141234,iPhone,1,700.00,2019-01-22 21:25:00,"944 Walnut St, Boston, MA 02215",MA,Boston,21,22,1
1,141235,Lightning Charging Cable,1,14.95,2019-01-28 14:15:00,"185 Maple St, Portland, OR 97035",OR,Portland,14,28,1
2,141236,Wired Headphones,2,11.99,2019-01-17 13:33:00,"538 Adams St, San Francisco, CA 94016",CA,San Francisco,13,17,1
3,141237,27in FHD Monitor,1,149.99,2019-01-05 20:33:00,"738 10th St, Los Angeles, CA 90001",CA,Los Angeles,20,5,1
4,141238,Wired Headphones,1,11.99,2019-01-25 11:59:00,"387 10th St, Austin, TX 73301",TX,Austin,11,25,1


In [ ]:
# Muestro el down del df
dataframe_e.tail()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Estado,Ciudad,Hora,Dia,Mes
9718,150497,20in Monitor,1,109.99,2019-01-26 19:09:00,"95 8th St, Dallas, TX 75001",TX,Dallas,19,26,1
9719,150498,27in FHD Monitor,1,149.99,2019-01-10 22:58:00,"403 7th St, San Francisco, CA 94016",CA,San Francisco,22,10,1
9720,150499,ThinkPad Laptop,1,999.99,2019-01-21 14:31:00,"214 Main St, Portland, OR 97035",OR,Portland,14,21,1
9721,150500,AAA Batteries (4-pack),2,2.99,2019-01-15 14:21:00,"810 2nd St, Los Angeles, CA 90001",CA,Los Angeles,14,15,1
9722,150501,Google Phone,1,600.00,2019-01-13 16:43:00,"428 Cedar St, Boston, MA 02215",MA,Boston,16,13,1


In [ ]:
dataframes['Febrero']

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Estado,Ciudad,Hora,Dia,Mes
0,150502,iPhone,1,700.00,2019-02-18 01:35:00,"866 Spruce St, Portland, ME 04101",ME,Portland,1,18,2
1,150503,AA Batteries (4-pack),1,3.84,2019-02-13 07:24:00,"18 13th St, San Francisco, CA 94016",CA,San Francisco,7,13,2
2,150504,27in 4K Gaming Monitor,1,389.99,2019-02-18 09:46:00,"52 6th St, New York City, NY 10001",NY,New York City,9,18,2
3,150505,Lightning Charging Cable,1,14.95,2019-02-02 16:47:00,"129 Cherry St, Atlanta, GA 30301",GA,Atlanta,16,2,2
4,150506,AA Batteries (4-pack),2,3.84,2019-02-28 20:32:00,"548 Lincoln St, Seattle, WA 98101",WA,Seattle,20,28,2
...,...,...,...,...,...,...,...,...,...,...,...
12031,162004,Apple Airpods Headphones,1,150.00,2019-02-12 22:02:00,"227 Church St, San Francisco, CA 94016",CA,San Francisco,22,12,2
12032,162005,AAA Batteries (4-pack),2,2.99,2019-02-04 20:44:00,"417 Jefferson St, Los Angeles, CA 90001",CA,Los Angeles,20,4,2
12033,162006,USB-C Charging Cable,1,11.95,2019-02-24 06:31:00,"498 8th St, Atlanta, GA 30301",GA,Atlanta,6,24,2
12034,162007,USB-C Charging Cable,1,11.95,2019-02-24 19:09:00,"715 7th St, Dallas, TX 75001",TX,Dallas,19,24,2


In [ ]:
df_diciembre = dataframes['Diciembre'].sort_values('Dia', ascending=False)
df_diciembre.head()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Estado,Ciudad,Hora,Dia,Mes
21254,315968,AA Batteries (4-pack),3,3.84,2019-12-31 11:46:00,"887 Dogwood St, San Francisco, CA 94016",CA,San Francisco,11,31,12
9138,304377,27in 4K Gaming Monitor,1,389.99,2019-12-31 16:01:00,"529 12th St, Atlanta, GA 30301",GA,Atlanta,16,31,12
21507,316211,Bose SoundSport Headphones,1,99.99,2019-12-31 14:56:00,"17 Meadow St, Dallas, TX 75001",TX,Dallas,14,31,12
1933,297500,AA Batteries (4-pack),1,3.84,2019-12-31 14:24:00,"637 Main St, Los Angeles, CA 90001",CA,Los Angeles,14,31,12
23855,318455,Apple Airpods Headphones,1,150.00,2019-12-31 21:06:00,"158 Pine St, Austin, TX 73301",TX,Austin,21,31,12


In [ ]:
dataframes['Enero']

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Estado,Ciudad,Hora,Dia,Mes
0,141234,iPhone,1,700.00,2019-01-22 21:25:00,"944 Walnut St, Boston, MA 02215",MA,Boston,21,22,1
1,141235,Lightning Charging Cable,1,14.95,2019-01-28 14:15:00,"185 Maple St, Portland, OR 97035",OR,Portland,14,28,1
2,141236,Wired Headphones,2,11.99,2019-01-17 13:33:00,"538 Adams St, San Francisco, CA 94016",CA,San Francisco,13,17,1
3,141237,27in FHD Monitor,1,149.99,2019-01-05 20:33:00,"738 10th St, Los Angeles, CA 90001",CA,Los Angeles,20,5,1
4,141238,Wired Headphones,1,11.99,2019-01-25 11:59:00,"387 10th St, Austin, TX 73301",TX,Austin,11,25,1
...,...,...,...,...,...,...,...,...,...,...,...
9718,150497,20in Monitor,1,109.99,2019-01-26 19:09:00,"95 8th St, Dallas, TX 75001",TX,Dallas,19,26,1
9719,150498,27in FHD Monitor,1,149.99,2019-01-10 22:58:00,"403 7th St, San Francisco, CA 94016",CA,San Francisco,22,10,1
9720,150499,ThinkPad Laptop,1,999.99,2019-01-21 14:31:00,"214 Main St, Portland, OR 97035",OR,Portland,14,21,1
9721,150500,AAA Batteries (4-pack),2,2.99,2019-01-15 14:21:00,"810 2nd St, Los Angeles, CA 90001",CA,Los Angeles,14,15,1


## Analisis y graficos de datos

## 1. Comportamiento de las ventas en los distintos meses:

  ● ¿Cómo variaron las ventas a lo largo de los diferentes meses?

  ● ¿Hubo algún mes que se destacó en términos de ventas?

  ● ¿Cuál es el ingreso total generado por mes?

In [ ]:
# Creo una lista para almacenar los totales de ventas por mes
totales_ventas = []
unidades_vendidas = []
# Itero los meses
for mes in meses:
    # Obtén el DataFrame del mes actual
    dataframe_mes = dataframes[mes]
    total_ventas_mes = dataframe_mes['Cantidad Pedida'].count()
    unidades_vendidas_mes =  dataframe_mes['Cantidad Pedida'].sum()
    # Agrega el total de ventas a la lista
    totales_ventas.append(total_ventas_mes)
    unidades_vendidas.append(unidades_vendidas_mes)

# Ahora, totales_ventas contendrá los totales de ventas en el mismo orden que la lista de meses
print("Meses:", meses)
print("Totales de Ventas:", totales_ventas)


Meses: ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
Totales de Ventas: [9681, 11986, 15154, 18289, 16554, 13556, 14291, 11957, 11629, 20284, 17580, 24989]


In [ ]:
# Creo la figura con plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(
    x=meses,
    y=unidades_vendidas,
    yaxis="y",
    name="Unidades vendidas",
    marker =dict(color=unidades_vendidas,colorscale= 'RdYlGn', colorbar= dict(title= "Total de ventas")),
    text = unidades_vendidas,
    textposition = 'auto'

))

fig.update_layout(
    title="Unidades vendidas por mes",
    title_x=0.5,
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Unidades vendidas", side = "left")
)

fig.show()

In [ ]:

fig = go.Figure()

fig.add_trace(go.Bar(
    x=meses,
    y=totales_ventas,
    yaxis="y",
    name="Ventas totales",
    marker =dict(color=totales_ventas,colorscale= 'RdYlGn', colorbar= dict(title= "Total de ventas")),
    text = totales_ventas,
    textposition = 'auto'

))

fig.update_layout(
    title="Ventas mensuales",
    title_x=0.5,
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Ventas totales", side = "left")
)

fig.show()

In [ ]:
import plotly.graph_objects as go

sizes = []
colors = []

#Hago un for para recorrer la cantidad total de ventas, y voy a dividir por una constante(1000), para hacer las burbujas de color acorde a la cantidad de unidades vendidas por mes
for total in totales_ventas:
  s = 35
  c = total / 1000
  sizes.append(s)
  colors.append(c)

fig = go.Figure(data=[go.Scatter(
    x=meses,
    y=totales_ventas,
    mode='markers',
    marker=dict(
        color=colors,
        size= sizes,
        showscale=True,
        colorbar= dict(title= "Ventas en miles")
        )
)])

fig.update_layout(

  title="Unidades vendidas por mes",
  title_x=0.5,
  xaxis=dict(title="Mes", tickvals=meses),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
  yaxis=dict(title="Unidades vendidas", side="left"),
  legend=dict(y=0.3, yanchor='bottom')
)


fig.show()

In [ ]:
ingresos_totales = []

for mes in meses:
  total_ventas_mes = (dataframes[mes]['Cantidad Pedida']*dataframes[mes]['Precio Unitario']).sum().round(2)
  ingresos_totales.append(total_ventas_mes)

# Crear el objeto figura
fig = go.Figure()

#Configuro titulos y ejes del grafico
fig.add_trace(go.Bar(
        x = meses,
        y = ingresos_totales,
        yaxis= 'y',
        name = 'Ingreso Total',
        marker = dict(color= ingresos_totales, colorscale='RdYlGn', colorbar=dict(title = 'Total de Ingresos')),
        text = ingresos_totales,
        textposition = 'outside',
)
)

fig.update_layout(
    title = 'Ingresos totales por mes',
    title_x=0.5,
    xaxis = dict(title='Mes'),
    yaxis = dict(title='Ingreso total',side='left')
)

#Muestro el grafico
fig.show()

## 2. Optimización de la publicidad y patrón de ventas por hora:

  ● ¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la
  probabilidad de compra?

  ● ¿Cómo cambian los patrones de ventas por hora a lo largo del año?

  ● ¿Hay modificaciones en los patrones de ventas durante las horas de mayor actividad
  en los distintos meses?

In [ ]:
# Creo el objeto figura
Horas = list(range(24))
fig = go.Figure()

# Voy a dibujar una hora por cada mes
for mes in meses:
  # Calculo el total de ventas agrupado en horas
  Ventas_por_hora= dataframes[mes].groupby('Hora')['Cantidad Pedida'].count()

  # Agrego al objeto figura un grafico de de linea con las horas y las ventas por hora
  fig.add_trace(go.Scatter(
      x=Horas,
      y=Ventas_por_hora,
      mode='lines',
      name=mes,
      line=dict(shape='spline', smoothing=1.3, width=2)))


  # Configuro los ejes y los títulos
fig.update_layout(

    title="Unidades vendidas por hora a lo largo de cada mes",
    title_x = 0.5,
    xaxis=dict(title="Hora", tickvals=Horas),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom')),



fig.show()

In [ ]:
fig = go.Figure()

horas_analizadas = [12,19]

for hora in horas_analizadas:
  total_productos_vendidos = []

  for mes in meses:
    dataframe_mes = dataframes[mes]
    ventas_hora = dataframe_mes[dataframe_mes['Hora'] == hora]

    total_productos_vendidos.append(ventas_hora['Cantidad Pedida'].count())

  fig.add_trace(go.Scatter(

      x=meses,
      y=total_productos_vendidos,
      mode='lines',
      name=f'{hora} horas',
      line = dict(width= 2, shape= 'spline')
  ))


# Configuro los ejes y los títulos
fig.update_layout(

  title="Unidades vendidas por hora en horas de mayor actividad",
  title_x=0.5,
  xaxis=dict(title="Hora", tickvals=Horas),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
  yaxis=dict(title="Unidades vendidas", side="left"),
  legend=dict(y=0.3, yanchor='bottom')
)


fig.show()

 Optimizacion de las ventas por hora


In [ ]:
ventas_hora = []
# Agrupo las ventas por hora y calculo el total de ventas por hora
for mes in meses:
  ventas_por_hora = dataframes[mes].groupby(dataframes[mes]['Hora'])['Cantidad Pedida'].count().reset_index()
  ventas_hora.append(ventas_por_hora)
# Encuentro la hora con mas ventas
hora_max_ventas = ventas_por_hora.loc[ventas_por_hora['Cantidad Pedida'].idxmax()]['Hora']
print(f"La hora ideal para presentar la publicidad es a las {hora_max_ventas} horas.")

La hora ideal para presentar la publicidad es a las 19 horas.


## 3. Distribución de ventas por ubicación:

  ● ¿En qué ciudades se han registrado las mayores ventas?

  ● ¿Cómo se comparan las ventas en diferentes estados o regiones?

  ● ¿Existe variación en las ventas por estado a lo largo de los meses?


In [6]:
# Inicio un DataFrame vacio para almacenar las ventas por ciudad de todos los meses
ventas_por_ciudad = pd.DataFrame()

for mes in meses:
    # Realizo el calculo de las ventas por ciudad para el mes actual
    ventas_mes = dataframes[mes].groupby('Ciudad')['Cantidad Pedida'].count().reset_index(inplace=False)
    # Concateo el DataFrame del mes actual al  general
    ventas_por_ciudad = pd.concat([ventas_por_ciudad, ventas_mes], ignore_index=True)

ventas_por_ciudad

,Ciudad,Cantidad Pedida
0,Atlanta,786
1,Austin,527
2,Boston,1048
3,Dallas,785
4,Los Angeles,1529
...,...,...
103,Los Angeles,3894
104,New York City,3408
105,Portland,1648
106,San Francisco,6021


In [7]:
# Agrupo por  'Ciudad' y suma la columna 'Cantidad Pedida' pero como anteriormente use count para obtener el total de ventas y no la cantidad de productos,
# esto me esta mostrando la cantidad de ventas que hubo (no la cantidad de productos vendidos)
ventas_por_ciudad = ventas_por_ciudad.groupby('Ciudad')['Cantidad Pedida'].sum().reset_index()
ventas_por_ciudad

,Ciudad,Cantidad Pedida
0,Atlanta,14881
1,Austin,9905
2,Boston,19934
3,Dallas,14820
4,Los Angeles,29605
5,New York City,24876
6,Portland,12465
7,San Francisco,44732
8,Seattle,14732


In [9]:
# Calculo los maximos por ciudad y los guardo en una variable
maximos_por_ciudad = ventas_por_ciudad.groupby('Ciudad')['Cantidad Pedida'].max().reset_index()
# Obtengo una paleta de colores similar a la usada en Plotly Express
colores_ciudad = px.colors.qualitative.Set1

# Creo un diccionario que mapee cada ciudad a su color correspondiente
color_por_ciudad = {}
ciudades = ventas_por_ciudad['Ciudad']
cantidades = ventas_por_ciudad['Cantidad Pedida']
for i, ciudad in enumerate(ciudades):
    color = colores_ciudad[i % len(colores_ciudad)]  # Ciclar colores de la paleta
    color_por_ciudad[ciudad] = color



In [11]:
# Creo lag figura
fig = go.Figure()

# Agrego el grafico de lineas
fig.add_trace(go.Scatter(
    x=ciudades,
    y=cantidades,
    mode='lines',
    name="Variacion de ventas",
    line=dict(width=2, shape='spline')
))

# Marcolos puntos maximos por ciudad
for _, row in maximos_por_ciudad.iterrows():
    ciudad_max = row['Ciudad']
    cantidad_max = row['Cantidad Pedida']
    color_max = color_por_ciudad[ciudad_max]  # Obteng el color correspondiente a la ciudad maxima
    fig.add_trace(go.Scatter(
        x=[ciudad_max],
        y=[cantidad_max],
        mode='markers',
        name=f'Máximo ({ciudad_max})',
        marker=dict(size=10, color=color_max)  # Asigno el color al grafico
    ))

# Configurar ejes y título
fig.update_layout(
    title="Ventas anuales por ciudad",
    title_x=0.5,
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)
fig.show()

In [ ]:

fig = go.Figure()

# Agregar el gráfico de dispersión con los máximos por ciudad
for _, row in maximos_por_ciudad.iterrows():
    ciudad_max = row['Ciudad']
    cantidad_max = row['Cantidad Pedida']
    color = "red"
# Agrego el grafico de barras con colores por ciudad
for ciudad in ciudades:
    color = color_por_ciudad[ciudad]  # Obtengo el color correspondiente a la ciudad
    fig.add_trace(go.Bar(
        x=[ciudad],
        y=[cantidades[ciudades == ciudad].values[0]],  # Obtengo la cantidad para la ciudad especifica
        name=ciudad,
        marker_color=color  # Asigno el color al grafico de barras
    ))
# Configuro ejes y titulo
fig.update_layout(
    title="Ventas anuales por ciudad",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)

fig.show()


In [15]:
# Voy a crear un grafico pastel para mostrar los porcentajes de venta por cada ciudad
total_ventas = cantidades.count()
proporciones = cantidades / total_ventas
indice_ciudad_max_ventas = proporciones.idxmax()

# Creo un DataFrame para el grafico ya que para el grafico de pie necesito un dataframe con las proporciones de ventas
df_pie = pd.DataFrame({'Ciudad': ciudades, 'Ventas': cantidades, 'Proporción': proporciones})

# Creo el grafico de torta interactivo
fig = px.pie(df_pie, names='Ciudad', values='Ventas', color='Ciudad', color_discrete_map=color_por_ciudad,
             labels={'Ciudad': 'Ciudad', 'Ventas': 'Ventas'},
             title='Porcentaje de Ventas por Ciudad')

# Agrego las etiquetas de porcentaje
fig.update_traces(textinfo='percent+label')

# Destaco la ciudad con mas ventas
pull_values = [0] * len(ciudades)
pull_values[indice_ciudad_max_ventas] = 0.1
fig.update_traces(pull=pull_values)

# Configuro el diseño de la leyenda y el titulo
fig.update_layout(
    legend_title_text='Ciudades',
    title_text='Porcentaje de Ventas por Ciudad',
    title_x=0.5,  # Centra el título horizontalmente
)
# Ploteo
fig.show()


In [71]:
# Ahora voy a investigar las ventas totales por estado
# Inicio un DataFrame vacio para almacenar las ventas por ciudad de todos los meses
ventas_por_estado = pd.DataFrame()

for mes in meses:
    # Realizo el calculo de las ventas por ciudad para el mes actual
    ventas_mes = dataframes[mes].groupby('Estado')['Cantidad Pedida'].count().reset_index(inplace=False)
    # Concateo el DataFrame del mes actual al  general
    ventas_por_estado = pd.concat([ventas_por_estado, ventas_mes], ignore_index=True)

ventas_por_estado

,Estado,Cantidad Pedida
0,CA,3883
1,GA,786
2,MA,1048
3,ME,123
4,NY,1303
...,...,...
91,ME,315
92,NY,3408
93,OR,1333
94,TX,3355


In [72]:
# Agrupo por  'Estado' y suma la columna 'Cantidad Pedida' pero como anteriormente use count para obtener el total de ventas y no la cantidad de productos,
# esto me esta mostrando la cantidad de ventas que hubo (no la cantidad de productos vendidos)
ventas_por_estado = ventas_por_estado.groupby('Estado')['Cantidad Pedida'].sum().reset_index()
ventas_por_estado

,Estado,Cantidad Pedida
0,CA,74337
1,GA,14881
2,MA,19934
3,ME,2455
4,NY,24876
5,OR,10010
6,TX,24725
7,WA,14732


In [61]:
estados = ventas_por_estado['Estado']
cantidad = ventas_por_estado['Cantidad Pedida']

In [62]:
# Calcular los máximos por estado
maximos_por_estado = ventas_por_estado.groupby('Estado')['Cantidad Pedida'].max().reset_index()

In [73]:
import plotly.graph_objects as go

# Crear un gráfico de barras
fig = go.Figure()
for _, row in maximos_por_estado.iterrows():
    estado_max = row['Estado']
    cantidad_max = row['Cantidad Pedida']
    fig.add_trace(go.Bar(
        x=[estado_max],
        y=[cantidad_max],
        name=estado_max,
        text=cantidad_max
    ))

# Configurar ejes y título
fig.update_layout(
    title="Ventas por estado",
    title_x=0.5,
    xaxis=dict(title="Estado"),
    yaxis=dict(title="Ventas"),
)

fig.show()


In [33]:
import plotly.graph_objs as go


#maximos_por_estado = maximos_por_estado.sort_values(by='Cantidad Pedida', ascending=True)
# Crear el gráfico
fig = go.Figure()



# Agregar el gráfico de líneas
fig.add_trace(go.Scatter(
    x=estados,
    y=cantidad,
    mode='lines',
    name="Ciudad",
    line=dict(width=2, shape='spline')
))

# Marcar los puntos máximos por estado
for _, row in maximos_por_estado.iterrows():
    estado_max = row['Estado']
    cantidad_max = row['Cantidad Pedida']
    fig.add_trace(go.Scatter(
        x=[estado_max],
        y=[cantidad_max],
        mode='markers',
        name=f'Máximo ({estado_max})',
        marker=dict(size=10)
    ))

# Configurar ejes y título
fig.update_layout(
    title="Unidades vendidas por estado",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)

fig.show()

In [19]:
import plotly.express as px


# Hago un plot de Estados Unidos
fig = px.choropleth(ventas_por_estado, locations='Estado', locationmode='USA-states',color='Cantidad Pedida',
                           color_continuous_scale="RdYlGn",
                           scope="usa",
                          title= 'Ventas por estado'
                          )
fig.update_layout(
    title="Ventas anuales por Estado",
    title_x=0.5,
)
fig.show()

In [75]:
# Voy a graficar las ventas por mes, en cada estado
ventas_por_estado_mes = {}
fig = go.Figure()

for mes in meses:
  ventas_por_estado_mes[mes] = dataframes[mes].groupby('Estado')['Cantidad Pedida'].count()
  ventas_por_estado_mes[mes] = ventas_por_estado_mes[mes].reset_index(inplace=False)

# Agrego al objeto figura un grafico de barras
  fig.add_trace(go.Bar(
      x=ventas_por_estado_mes[mes]['Estado'],
      y=ventas_por_estado_mes[mes]['Cantidad Pedida'],
      name= mes,
  ));

# Configuro ejes y los titulos
fig.update_layout(
    title="Total de ventas por mes en cada estado",
    title_x=0.5,
    xaxis=dict(title="Estados"),
    yaxis=dict(title="Ventas", side="left"),
);

fig.show()
estados

0    CA
1    GA
2    MA
3    ME
4    NY
5    OR
6    TX
7    WA
Name: Estado, dtype: object

## 4. Análisis del producto más vendido:
  ● ¿Cuál es el producto más vendido en general y en cada mes?

  ● ¿Qué factores crees que han contribuido al éxito de ese producto en particular?

In [ ]:
# Creo un DataFrame general combinando con todos los meses
df_4 = pd.concat(dataframes.values(), ignore_index=True)

# Calculo las ventas totales por producto
producto_mas_vendido_general = df_4.groupby('Producto')['Cantidad Pedida'].sum().idxmax()
total_ventas_producto_mas_vendido = df_4.groupby('Producto')['Cantidad Pedida'].sum().max()

print("Producto más vendido en general:", producto_mas_vendido_general)
print("Total de ventas del producto más vendido:", total_ventas_producto_mas_vendido)
df_4.head()

Producto más vendido en general: AAA Batteries (4-pack)
Total de ventas del producto más vendido: 31017


,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Estado,Ciudad,Hora,Dia,Mes
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",TX,Dallas,8,19,4
1,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",MA,Boston,22,7,4
2,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",CA,Los Angeles,14,12,4
3,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",CA,Los Angeles,14,12,4
4,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",CA,Los Angeles,9,30,4


In [ ]:
# Calcular las ventas totales por producto en el DataFrame combinado
ventas_totales = df_4.groupby('Producto')['Cantidad Pedida'].sum().reset_index()

# Crear un gráfico de barras para mostrar las ventas totales por producto
fig = go.Figure()

fig.add_trace(go.Bar(
    x=ventas_totales['Producto'],
    y=ventas_totales['Cantidad Pedida'],
    yaxis="y",
    name="Unidades vendidas",
    marker=dict(color=ventas_totales['Cantidad Pedida'], colorscale='RdYlGn', colorbar=dict(title="Total de ventas")),
    text=ventas_totales['Cantidad Pedida'],
    textposition='auto'
))

fig.update_layout(
    title="Ventas totales de todos los productos en general",
    xaxis=dict(title="Producto"),
    yaxis=dict(title="Unidades vendidas", side="left")
)

fig.show()

In [ ]:
# Voy a hacer un grafico que me muestre la venta de cada producto a lo largo delos difrentes meses
# Creo una lista de DataFrames mensuales

# Ordenar el DataFrame por la columna 'Mes' categorica
df_4 = df_4.sort_values(by='Mes')
df_4.head()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Estado,Ciudad,Hora,Dia,Mes
58477,144331,Apple Airpods Headphones,1,150.00,2019-01-26 14:08:00,"389 Hill St, Los Angeles, CA 90001",CA,Los Angeles,14,26,1
61673,147385,Apple Airpods Headphones,1,150.00,2019-01-20 20:09:00,"310 Adams St, San Francisco, CA 94016",CA,San Francisco,20,20,1
61674,147386,USB-C Charging Cable,1,11.95,2019-01-01 11:44:00,"793 11th St, San Francisco, CA 94016",CA,San Francisco,11,1,1
61675,147387,USB-C Charging Cable,1,11.95,2019-01-24 21:07:00,"753 Pine St, Austin, TX 73301",TX,Austin,21,24,1
61676,147388,AA Batteries (4-pack),1,3.84,2019-01-20 12:55:00,"91 8th St, San Francisco, CA 94016",CA,San Francisco,12,20,1


In [ ]:
df_4.tail()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Estado,Ciudad,Hora,Dia,Mes
46931,311693,27in FHD Monitor,1,149.99,2019-12-06 18:05:00,"594 9th St, Dallas, TX 75001",TX,Dallas,18,6,12
46932,311694,AAA Batteries (4-pack),1,2.99,2019-12-19 07:35:00,"905 11th St, Atlanta, GA 30301",GA,Atlanta,7,19,12
46933,311695,Wired Headphones,1,11.99,2019-12-21 21:06:00,"107 Lakeview St, San Francisco, CA 94016",CA,San Francisco,21,21,12
46911,311673,USB-C Charging Cable,1,11.95,2019-12-26 22:24:00,"169 Forest St, San Francisco, CA 94016",CA,San Francisco,22,26,12
43290,308183,Lightning Charging Cable,1,14.95,2019-12-11 13:25:00,"270 8th St, Los Angeles, CA 90001",CA,Los Angeles,13,11,12


In [ ]:
# Conviero la columna 'Mes' en categorica con el orden deseado usando la lista de meses
df_4['MesName'] = pd.Categorical(df_4['Mes'].map(lambda x: meses[x-1]), categories=meses, ordered=True)
df_4.head()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Estado,Ciudad,Hora,Dia,Mes,MesName
58477,144331,Apple Airpods Headphones,1,150.00,2019-01-26 14:08:00,"389 Hill St, Los Angeles, CA 90001",CA,Los Angeles,14,26,1,Enero
61673,147385,Apple Airpods Headphones,1,150.00,2019-01-20 20:09:00,"310 Adams St, San Francisco, CA 94016",CA,San Francisco,20,20,1,Enero
61674,147386,USB-C Charging Cable,1,11.95,2019-01-01 11:44:00,"793 11th St, San Francisco, CA 94016",CA,San Francisco,11,1,1,Enero
61675,147387,USB-C Charging Cable,1,11.95,2019-01-24 21:07:00,"753 Pine St, Austin, TX 73301",TX,Austin,21,24,1,Enero
61676,147388,AA Batteries (4-pack),1,3.84,2019-01-20 12:55:00,"91 8th St, San Francisco, CA 94016",CA,San Francisco,12,20,1,Enero


In [ ]:
# Calculo las ventas totales por producto y mes
ventas_totales_por_producto_mes = df_4.groupby(['Producto', 'MesName'])['Cantidad Pedida'].sum().reset_index()

# Creo un grafico de barras para mostrar las ventas totales por producto y mes
fig_ventas_por_mes = go.Figure()

for producto in ventas_totales_por_producto_mes['Producto'].unique():
    datos_producto = ventas_totales_por_producto_mes[ventas_totales_por_producto_mes['Producto'] == producto]
    fig_ventas_por_mes.add_trace(go.Bar(
        x=datos_producto['MesName'],
        y=datos_producto['Cantidad Pedida'],
        name=producto
    ))

fig_ventas_por_mes.update_layout(
    title='Ventas totales de todos los productos por mes',
    xaxis=dict(title='Mes'),
    yaxis=dict(title='Cantidad Pedida')
)

fig_ventas_por_mes.show()

In [ ]:
# Creo un grafico de lineas para mostrar lo mismo
fig_ventas_por_mes = px.line(ventas_totales_por_producto_mes, x='MesName', y='Cantidad Pedida', color='Producto',
                             title='Ventas totales de todos los productos por mes')

fig_ventas_por_mes.update_xaxes(type='category')  # Establecer el eje X como categórico (meses)
fig_ventas_por_mes.show()

In [ ]:
# Ahora voy a analizar el ingreso que generan los productos, para ver la relacion que guarda con el producto mas vendido
# Creo una nueva columna 'Ingreso' que sea el resultado de multiplicar 'Cantidad Pedida' por 'Precio Unitario'
df_4['Ingreso'] = df_4['Cantidad Pedida'] * df_4['Precio Unitario']

# Agrupar por producto y calcular los ingresos totales por producto
ingresos_por_producto = df_4.groupby('Producto')['Ingreso'].sum()

# Encontrar el producto con el mayor ingreso
producto_mas_ingresos = ingresos_por_producto.idxmax()
total_ingresos_producto_mas_ingresos = ingresos_por_producto.max()

print("Producto que genera más ingresos:", producto_mas_ingresos)
print("Total de ingresos del producto más rentable:", total_ingresos_producto_mas_ingresos)


Producto que genera más ingresos: Macbook Pro Laptop
Total de ingresos del producto más rentable: 8037600.0


In [ ]:
# Calculo los ingresos totales por producto en el DataFrame combinado
ingresos_totales = df_4.groupby('Producto')['Ingreso'].sum().round().reset_index()

# Creo un graafico de barras para mostrar los ingresos totales por producto
fig = go.Figure()

fig.add_trace(go.Bar(
    x=ingresos_totales['Producto'],
    y=ingresos_totales['Ingreso'],
    yaxis="y",
    name="Ingresos generados",
    marker=dict(color=ingresos_totales['Ingreso'], colorscale='RdYlGn', colorbar=dict(title="Total de ingresos")),
    text=ingresos_totales['Ingreso'],
    textposition='auto'
))

fig.update_layout(
    title="Ingresos totales generados por cada producto",
    title_x = 0.5,
    xaxis=dict(title="Producto"),
    yaxis=dict(title="Ingresos generados", side="left")
)

fig.show()

In [ ]:
# Calculo los porcentajes de ingresos
ingresos_totales['Porcentaje'] = ingresos_totales['Ingreso'] / ingresos_totales['Ingreso'].sum() * 100
indice_max_ingresos = ingresos_totales['Ingreso'].idxmax()
# Creo un grafico de pastel interactivo
fig = px.pie(ingresos_totales, names='Producto', values='Porcentaje',
             labels={'Producto': 'Producto', 'Porcentaje': 'Porcentaje'},
             title='Porcentaje de Ingresos por Producto')

# Ajusto el tamaño del grafico
fig.update_layout(width=1500, height=800)

# Muestro los porcentajes dentro del grafico
fig.update_traces(textinfo='percent+label')

# Destaco el producto que genera mas
fig.update_traces(pull=[0.1 if i == indice_max_ingresos else 0 for i in range(len(ingresos_totales))])

fig.update_layout(legend_title_text='Productos')

fig.show()

## 5. Tendencia ventas:

  ● ¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días
  en los diferentes meses?

  ● ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

  ● ¿Se observa alguna diferencia significativa en las ventas entre los días laborables y
  los fines de semana?

In [ ]:
# Creo un df con todos los meses juntos
df_5 = pd.concat(dataframes.values(), ignore_index=True)
df_5 = df_5.sort_values(by='Mes')
df_5.head()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Estado,Ciudad,Hora,Dia,Mes
58477,144331,Apple Airpods Headphones,1,150.00,2019-01-26 14:08:00,"389 Hill St, Los Angeles, CA 90001",CA,Los Angeles,14,26,1
61673,147385,Apple Airpods Headphones,1,150.00,2019-01-20 20:09:00,"310 Adams St, San Francisco, CA 94016",CA,San Francisco,20,20,1
61674,147386,USB-C Charging Cable,1,11.95,2019-01-01 11:44:00,"793 11th St, San Francisco, CA 94016",CA,San Francisco,11,1,1
61675,147387,USB-C Charging Cable,1,11.95,2019-01-24 21:07:00,"753 Pine St, Austin, TX 73301",TX,Austin,21,24,1
61676,147388,AA Batteries (4-pack),1,3.84,2019-01-20 12:55:00,"91 8th St, San Francisco, CA 94016",CA,San Francisco,12,20,1


In [ ]:
# Voy a obtener el nombre de cada mes de la semana y lo guardo en una columna nueva
df_5['MesName'] = pd.Categorical(df_5['Mes'].map(lambda x: meses[x-1]), categories=meses, ordered=True)
df_5.head()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Estado,Ciudad,Hora,Dia,Mes,MesName
58477,144331,Apple Airpods Headphones,1,150.00,2019-01-26 14:08:00,"389 Hill St, Los Angeles, CA 90001",CA,Los Angeles,14,26,1,Enero
61673,147385,Apple Airpods Headphones,1,150.00,2019-01-20 20:09:00,"310 Adams St, San Francisco, CA 94016",CA,San Francisco,20,20,1,Enero
61674,147386,USB-C Charging Cable,1,11.95,2019-01-01 11:44:00,"793 11th St, San Francisco, CA 94016",CA,San Francisco,11,1,1,Enero
61675,147387,USB-C Charging Cable,1,11.95,2019-01-24 21:07:00,"753 Pine St, Austin, TX 73301",TX,Austin,21,24,1,Enero
61676,147388,AA Batteries (4-pack),1,3.84,2019-01-20 12:55:00,"91 8th St, San Francisco, CA 94016",CA,San Francisco,12,20,1,Enero


In [ ]:
# Voy a obtener el nombre de cada dia de la semana y lo guardo en una columna nueva
df_5['DiaName'] = df_5['Fecha de Pedido'].dt.strftime('%A')

df_5.head()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Estado,Ciudad,Hora,Dia,Mes,MesName,DiaName
58477,144331,Apple Airpods Headphones,1,150.00,2019-01-26 14:08:00,"389 Hill St, Los Angeles, CA 90001",CA,Los Angeles,14,26,1,Enero,Saturday
61673,147385,Apple Airpods Headphones,1,150.00,2019-01-20 20:09:00,"310 Adams St, San Francisco, CA 94016",CA,San Francisco,20,20,1,Enero,Sunday
61674,147386,USB-C Charging Cable,1,11.95,2019-01-01 11:44:00,"793 11th St, San Francisco, CA 94016",CA,San Francisco,11,1,1,Enero,Tuesday
61675,147387,USB-C Charging Cable,1,11.95,2019-01-24 21:07:00,"753 Pine St, Austin, TX 73301",TX,Austin,21,24,1,Enero,Thursday
61676,147388,AA Batteries (4-pack),1,3.84,2019-01-20 12:55:00,"91 8th St, San Francisco, CA 94016",CA,San Francisco,12,20,1,Enero,Sunday


In [ ]:
df_5.tail()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Estado,Ciudad,Hora,Dia,Mes,MesName,DiaName
46931,311693,27in FHD Monitor,1,149.99,2019-12-06 18:05:00,"594 9th St, Dallas, TX 75001",TX,Dallas,18,6,12,Diciembre,Friday
46932,311694,AAA Batteries (4-pack),1,2.99,2019-12-19 07:35:00,"905 11th St, Atlanta, GA 30301",GA,Atlanta,7,19,12,Diciembre,Thursday
46933,311695,Wired Headphones,1,11.99,2019-12-21 21:06:00,"107 Lakeview St, San Francisco, CA 94016",CA,San Francisco,21,21,12,Diciembre,Saturday
46911,311673,USB-C Charging Cable,1,11.95,2019-12-26 22:24:00,"169 Forest St, San Francisco, CA 94016",CA,San Francisco,22,26,12,Diciembre,Thursday
43290,308183,Lightning Charging Cable,1,14.95,2019-12-11 13:25:00,"270 8th St, Los Angeles, CA 90001",CA,Los Angeles,13,11,12,Diciembre,Wednesday


In [ ]:
# Agrupo por mes y diaa y calculo la suma de las ventas diarias
ventas_diarias_por_mes = df_5.groupby(['Mes', 'Dia'])['Cantidad Pedida'].count().reset_index()


In [ ]:
# Grafico
import plotly.express as px

fig = px.scatter(ventas_diarias_por_mes, x='Dia', y='Cantidad Pedida', color='Mes', title='Ventas Diarias por Mes')
fig.show()


In [ ]:
# Creo un grafico de barras para cada mes
for mes in meses:
    # Filtrar el DataFrame para el mes actual
    df_mes = df_5[df_5['MesName'] == mes]

    # Calculo la suma de las ventas por dia para el mes actual
    ventas_diarias = df_mes.groupby('Dia')['Cantidad Pedida'].count()

    # Creo el grafico de barras
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=ventas_diarias.index,
        y=ventas_diarias.values,
        name=f'Ventas - Mes {mes}',
                marker=dict(
            color=ventas_diarias.values,  # Uso los valores de ventas como colores
            colorscale= 'RdYlGn',  # Especifico el colorscale
            colorbar=dict(title='Ventas'),  # Agrego barra de colores
        )
    ))

    # Configuro el diseño del grafico
    fig.update_layout(
        title=f'Ventas Diarias - Mes {mes}',
        xaxis_title='Día',
        yaxis_title='Ventas Diarias',
    )

    # Mostro el grafico
    fig.show()


In [ ]:

# Creo un grafico de subplots con 3 filas y 4 columnas
fig = make_subplots(rows=3, cols=4, subplot_titles=meses)

# Itero a traves de los meses y creo un grafico de barras para cada uno
for i, mes in enumerate(meses, 1):
    # Filtro el DataFrame para el mes
    df_mes = df_5[df_5['MesName'] == mes]

    # Calculo la suma de las ventas por dia para el mes actual
    ventas_diarias = df_mes.groupby('Dia')['Cantidad Pedida'].count()

    # Obtengo la fila y la columna del subplot actual
    row = (i - 1) // 4 + 1
    col = (i - 1) % 4 + 1

    # Agrego el grafico de barras al subplot correspondiente
    fig.add_trace(go.Bar(
        x=ventas_diarias.index,
        y=ventas_diarias.values,
        name=f'Ventas - Mes {mes}',
        marker=dict(
            color=ventas_diarias.values,  # Uso los valores de ventas como colores
            colorscale='RdYlGn',  # Especifico el colorscale
            showscale=True,  # Mostro la barra de colores
            colorbar=dict(
                title='Ventas',  # Título vacío
                tickvals=[ventas_diarias.min(), ventas_diarias.max()],  # Muestra solo valores minimo y max
                ticktext=['min', 'max'],  # Etiquetas personalizadas
            ),
        )
    ), row=row, col=col)

    # Configuro el diseño del grafico del subplot actual
    fig.update_xaxes(title_text='Dia', row=row, col=col, title_standoff=1)
    fig.update_yaxes(title_text='Ventas', row=row, col=col, range=[200, 1000])

# Configuro el diseño del gráfico general
fig.update_layout(
    title='Ventas Diarias por Mes',
    showlegend=False,  # Oculto la leyenda individual de cada mes
)

fig.show()



### ● ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?
### ● ¿Se observa alguna diferencia significativa en las ventas entre los días laborables y los fines de semana?

In [ ]:
ventas_por_dia_semana = df_5.groupby('DiaName')['Cantidad Pedida'].count().reset_index()
ventas_por_dia_semana

,DiaName,Cantidad Pedida
0,Friday,26247
1,Monday,26547
2,Saturday,26492
3,Sunday,26551
4,Thursday,26461
5,Tuesday,27175
6,Wednesday,26477


In [ ]:
orden_dias_semana = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Convierte la columna "DiaName" en categórica con el orden deseado
df_5['DiaOrdenados'] = pd.Categorical(df_5['DiaName'], categories=orden_dias_semana, ordered=True)

# Ordena el DataFrame por la columna "DiaName" en el orden especificado
df_5 = df_5.sort_values(by='DiaOrdenados')
df_5

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Estado,Ciudad,Hora,Dia,Mes,MesName,DiaName,DiaOrdenados
117643,174384,ThinkPad Laptop,1,999.99,2019-03-11 16:27:00,"203 North St, New York City, NY 10001",NY,New York City,16,11,3,Marzo,Monday,Monday
79196,225127,Lightning Charging Cable,1,14.95,2019-07-15 20:54:00,"904 13th St, Dallas, TX 75001",TX,Dallas,20,15,7,Julio,Monday,Monday
47771,312493,Apple Airpods Headphones,1,150.00,2019-12-02 18:16:00,"730 Chestnut St, Seattle, WA 98101",WA,Seattle,18,2,12,Diciembre,Monday,Monday
47770,312492,AAA Batteries (4-pack),1,2.99,2019-12-02 12:24:00,"611 Sunset St, San Francisco, CA 94016",CA,San Francisco,12,2,12,Diciembre,Monday,Monday
88286,233868,AA Batteries (4-pack),2,3.84,2019-07-01 10:04:00,"451 Park St, Dallas, TX 75001",TX,Dallas,10,1,7,Julio,Monday,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101178,219484,34in Ultrawide Monitor,1,379.99,2019-06-16 13:03:00,"688 Main St, San Francisco, CA 94016",CA,San Francisco,13,16,6,Junio,Sunday,Sunday
101175,219481,Lightning Charging Cable,1,14.95,2019-06-09 21:51:00,"524 5th St, Dallas, TX 75001",TX,Dallas,21,9,6,Junio,Sunday,Sunday
117564,174306,AAA Batteries (4-pack),2,2.99,2019-03-17 09:47:00,"651 Church St, Boston, MA 02215",MA,Boston,9,17,3,Marzo,Sunday,Sunday
165831,270666,Lightning Charging Cable,1,14.95,2019-10-13 17:56:00,"86 4th St, Austin, TX 73301",TX,Austin,17,13,10,Octubre,Sunday,Sunday


In [ ]:
# Creo un grafico de subplots con 3 filas y 4 columnas
fig = make_subplots(rows=3, cols=4, subplot_titles=meses, horizontal_spacing=0.1, vertical_spacing=0.25)

# Itero a traves de los meses y creo un grafico de barras para cada uno
for i, mes in enumerate(meses, 1):
    # Filtro el DataFrame para el mes
    df_mes = df_5[df_5['MesName'] == mes]

    # Calculo la suma de las ventas por dia para el mes actual
    ventas_diarias_por_semana = df_mes.groupby('DiaOrdenados')['Cantidad Pedida'].count()

    # Obtengo la fila y la columna del subplot actual
    row = (i - 1) // 4 + 1
    col = (i - 1) % 4 + 1

    # Agrego el grafico de barras al subplot correspondiente
    fig.add_trace(go.Bar(
        x=ventas_diarias_por_semana.index,
        y=ventas_diarias_por_semana.values,
        name=f'Ventas - Mes {mes}',
        marker=dict(
            color=ventas_diarias_por_semana.values,  # Uso los valores de ventas como colores
            colorscale='RdYlGn',  # Especifico el colorscale
            showscale=True,  # Mostro la barra de colores
            colorbar=dict(
                title='Ventas',  # Título vacío
                tickvals=[ventas_diarias_por_semana.min(), ventas_diarias_por_semana.max()],  # Muestra solo valores minimo y max
                ticktext=['min', 'max'],  # Etiquetas personalizadas
            ),
        )
    ), row=row, col=col)

    # Configuro el diseño del grafico del subplot actual
    fig.update_xaxes(row=row, col=col,title_standoff=1)
    fig.update_yaxes(title_text='Ventas', row=row, col=col, range=[0, 4400])

# Configuro el diseño del gráfico general
fig.update_layout(
    title='Ventas Diarias por Semana a lo largo de todos los meses',
    showlegend=False,  # Oculto la leyenda individual de cada mes
)

fig.show()

## 6. Impacto de eventos especiales en las ventas:

  ● ¿Se ha observado algún aumento o disminución significativa en las ventas en días
  cercanos a eventos especiales, como días festivos?

  ● ¿Qué eventos específicos han tenido un impacto notable en el comportamiento de
  las ventas y cómo se manifestó ese impacto?

In [ ]:
# Creo un df con todos los meses juntos
df_6 = pd.concat(dataframes.values(), ignore_index=True)
df_6 = df_6.sort_values(by='Mes')
df_6.head()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Estado,Ciudad,Hora,Dia,Mes
58477,144331,Apple Airpods Headphones,1,150.00,2019-01-26 14:08:00,"389 Hill St, Los Angeles, CA 90001",CA,Los Angeles,14,26,1
61673,147385,Apple Airpods Headphones,1,150.00,2019-01-20 20:09:00,"310 Adams St, San Francisco, CA 94016",CA,San Francisco,20,20,1
61674,147386,USB-C Charging Cable,1,11.95,2019-01-01 11:44:00,"793 11th St, San Francisco, CA 94016",CA,San Francisco,11,1,1
61675,147387,USB-C Charging Cable,1,11.95,2019-01-24 21:07:00,"753 Pine St, Austin, TX 73301",TX,Austin,21,24,1
61676,147388,AA Batteries (4-pack),1,3.84,2019-01-20 12:55:00,"91 8th St, San Francisco, CA 94016",CA,San Francisco,12,20,1


In [ ]:
fechas_especiales = [
    "2019-01-01",  # Año Nuevo
    "2019-02-12",  # Día de San Valentín
    "2019-02-13",  # Día de San Valentín
    "2019-02-14",  # Día de San Valentín
    "2019-04-19",   #Pascua
    "2019-04-20",   #Pascua
    "2019-04-21",   #Pascua
    "2019-05-09",  # Día de la Madre
    "2019-05-10",  # Día de la Madre
    "2019-05-11",  # Día de la Madre
    "2019-05-12",  # Día de la Madre
    "2019-06-06",  # Día del Niño
    "2019-06-07",  # Día del Niño
    "2019-06-08",  # Día del Niño
    "2019-06-09",  # Día del Niño
    "2019-06-14",  # Día del Padre
    "2019-06-15",  # Día del Padre
    "2019-06-16",  # Día del Padre
    "2019-07-04",  # Día de la Independencia (EE. UU.)
    "2019-09-04",  # Día del Trabajo (EE. UU.)
    "2019-10-28",  # Halloween
    "2019-10-29",  # Halloween
    "2019-10-30",  # Halloween
    "2019-10-31",  # Halloween
    "2019-11-08",  # Día de los Veteranos (EE. UU.)
    "2019-11-09",  # Día de los Veteranos (EE. UU.)
    "2019-11-10",  # Día de los Veteranos (EE. UU.)
    "2019-11-11",  # Día de los Veteranos (EE. UU.)
    "2019-11-21",  # Día de Acción de Gracias (EE. UU.)
    "2019-11-22",  # Día de Acción de Gracias (EE. UU.)
    "2019-11-23",  # Día de Acción de Gracias (EE. UU.)
    "2019-11-24",  # Día de Acción de Gracias (EE. UU.)

    "2019-11-29",  # Black friday
    "2019-11-30",  # Black friday
    "2019-12-01",  # Black friday
    "2019-12-02",  # Black friday
    "2019-12-03",  # Black friday

    "2019-12-21",   # Navidad
    "2019-12-22",   # Navidad
    "2019-12-23",   # Navidad
    "2019-12-24",   # Navidad
    "2019-12-25",   # Navidad
    "2019-12-29",   # Año nuevo
    "2019-12-30",   # Año nuevo
    "2019-12-31",  # Nochevieja

]


In [ ]:

df_6['Fecha de Pedido'] = pd.to_datetime(df_6['Fecha de Pedido'])
# Creo un DataFrame con las ventas por dia
ventas_por_dia = df_6.groupby(df_6['Fecha de Pedido'].dt.date)['Cantidad Pedida'].sum().reset_index()

# Cro una columna que indique si es festivo o no
ventas_por_dia['Es Festivo'] = ventas_por_dia['Fecha de Pedido'].astype(str).isin(fechas_especiales)

# Crea el grafico de barras
fig = px.bar(ventas_por_dia, x='Fecha de Pedido', y='Cantidad Pedida', title='Ventas Diarias en el Año')

# Personalizo los colores de las barras basados en si es festivo o no
colores = ['red' if es_festivo else 'blue' for es_festivo in ventas_por_dia['Es Festivo']]
fig.update_traces(marker_color=colores)

# Agrego un titulo al eje x
fig.update_layout(
    xaxis_title='Dias',  # Esto agrega un título al eje x.
    yaxis_title='Cantidad Pedida'  # Agrega el título al eje y.
)
fig.show()